### COLLOCATIONS

#### Отчет

Задача: На основании этих данных выделить наиболее типичные для суда действия (словосочетания типа принять решение), применив два различных метода.

Для оценки результата:

1) Пользуясь словарями, интуицией и т.п. составляете золотой стандарт – ранжированный топ 10 коллокаций:

> golden_stand = [(('ПРИНЯТЬ','РЕШЕНИЕ'),1), (('ПРИЗНАТЬ','ВИНОВНАЯ'),2), (('ВЫНЕСТИ','ПРИГОВОР'),3), 
                (('НАЗНАЧИТЬ','НАКАЗАНИЕ'),4), (('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'),5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'),6), (('ОТКЛОНИТЬ','ХОДАТАЙСТВО'),7), (('ОТКЛОНИТЬ','ИСК'),8), 
                (('ОБЖАЛОВАТЬ','ПРИГОВОР'),9), (('ПРЕКРАТИТЬ','ДЕЛО'),10)]


Методы:
* PMI
* LR

In [2]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [4]:
df = pd.read_csv('court-V-N.csv') # corpus
rows = list(map(list, df.values)) # list of rows in corpus
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)

In [22]:
golden_stand = [('ПРИНЯТЬ','РЕШЕНИЕ'), ('ПРИЗНАТЬ','ВИНОВНАЯ'), ('ВЫНЕСТИ','ПРИГОВОР'), 
                ('НАЗНАЧИТЬ','НАКАЗАНИЕ'), ('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'), 
                ('УДОВЛЕТВОРИТЬ','ИСК'), ('ОТКЛОНИТЬ','ХОДАТАЙСТВО'), ('ОТКЛОНИТЬ','ИСК'), 
                ('ОБЖАЛОВАТЬ','ПРИГОВОР'), ('ПРЕКРАТИТЬ','ДЕЛО')]

In [5]:
df.head()

,СУД,АРЕСТОВАТЬ,ПОДПОЛКОВНИК,ЗАМНАЧАЛЬНИК,ОТДЕЛ
0,СУД,ВЕСТИ,ПЕРЕГОВОРЫ,ПОСТАВКА,БУДУЩИЙ
1,СУД,ВЗЯТЬ,РЕАЛИЗАЦИЯ,ЗАКОН,ОБЯЗАТЕЛЬНЫЙ
2,СУД,ВЫКУПИТЬ,ГОСУДАРСТВО,АКЦИЯ,РАНЕЕ
3,СУД,ГОВОРИТЬ,ПРЕСС-СЕКРЕТАРЬ,ЕВГЕНИЙ,ВРЕМЕНИТЬ
4,СУД,ЕСТЬ,ЗВАНИЕ,ЗАСЛУЖЕННЫЙ,МАШИНОСТРОИТЕЛЬ


2) Смотрите, какую позицию занимают коллокации из золотого стандарта среди списка, отранжированного по мере коллокационной связи.

**Посмотрим как сотносятся коллокации из нашего золотого стандарта; выданные Pointwise Mutual Information и  Log-Likelihood**


In [27]:
bigrams_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)
bigrams_pmi = finder.nbest(bigram_measures.pmi, 10)

In [23]:
pd.DataFrame(data={'Golden standard': golden_stand, 'PMI': finder.nbest(bigram_measures.pmi, 10), 'LR': finder.nbest(bigram_measures.likelihood_ratio, 10)})

,Golden standard,LR,PMI
0,"(ПРИНЯТЬ, РЕШЕНИЕ)","(СУД , ПРИЗНАТЬ )","( АКЦИОНЕР , РОЗНИЧНЫЙ )"
1,"(ПРИЗНАТЬ, ВИНОВНАЯ)","(ВЫДАТЬ , САНКЦИЯ )","( АРМИЯ , ВОССТАНОВИТЬСЯ )"
2,"(ВЫНЕСТИ, ПРИГОВОР)","(СУД , УДОВЛЕТВОРИТЬ )","( БОЛЬШИНСТВО , СООТЕЧЕСТВЕННИК )"
3,"(НАЗНАЧИТЬ, НАКАЗАНИЕ)","(УДОВЛЕТВОРИТЬ , ИСК )","( БРИГАДИР , ВИКТОР )"
4,"(УДОВЛЕТВОРИТЬ, ХОДАТАЙСТВО)","(УДОВЛЕТВОРИТЬ , ХОДАТАЙСТВО )","( ВЕСТИ , ПЕРЕГОВОРЫ )"
5,"(УДОВЛЕТВОРИТЬ, ИСК)","(ПРИНЯТЬ , РЕШЕНИЕ )","( ВЗЯТЬ , РЕАЛИЗАЦИЯ )"
6,"(ОТКЛОНИТЬ, ХОДАТАЙСТВО)","(САНКЦИЯ , АРЕСТ )","( ВОЗГЛАВИТЬ , СПИСОК )"
7,"(ОТКЛОНИТЬ, ИСК)","(МЕРА , ПРЕСЕЧЕНИЕ )","( ВРЕМЯ , КНЯЗЕВ )"
8,"(ОБЖАЛОВАТЬ, ПРИГОВОР)","(НАЛОЖИТЬ , АРЕСТ )","( ВСПОМИНАТЬСЯ , ПОПЫТКА )"
9,"(ПРЕКРАТИТЬ, ДЕЛО)","(СУД , ВЫНЕСТИ )","( ДОЛЛАР , ПОТРАТИТЬ )"


(а) какие коллокации, полученные методом LR, не входят в мой золотой стандарт:

* (СУД , ПРИЗНАТЬ )	
* (САНКЦИЯ , АРЕСТ )	
* (МЕРА , ПРЕСЕЧЕНИЕ )	
* (НАЛОЖИТЬ , АРЕСТ )
* (СУД , ВЫНЕСТИ )	
* (ВЫДАТЬ , САНКЦИЯ )	

>Никакие из коллокаций, полученных методом PMI, не входят в мой золотой стандарт!

>**В целом кажется, что LR сильно лучше выделяет значимые коллокации, чем PMI**

(б) какие попали ошибочно (предложите объяснение - откуда взялась такая ошибка, что можно сделать, чтобы ее устранить)

>В целом кажется, что LR все определил адекватно.

>Наоборот, все коллокации, выделенные PMI, попали туда ошибочно)) Особенно "Бригадир Виктор". Может быть, если поставить ограничение на нижнюю границу частоты встречаемости, будет лучше.

(в) какие у Вас не попоали в ЗС, но вы думаете, что это полезные коллокации:

>(ВЫДАТЬ , САНКЦИЯ ) хорошая коллокация, не попавшая в мой ЗС

3) Считаете меру качества – ранговый коэффициент корреляции Спирмена между двумя списками.


In [29]:
# lists with scores and with scores removed
list_with_scores_lr = list(finder.score_ngrams(bigram_measures.likelihood_ratio))
list_no_scores_lr = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_lr]
list_with_scores_pmi = list(finder.score_ngrams(bigram_measures.pmi))
list_no_scores_pmi = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_pmi]

In [30]:
golden_stand = [(('ПРИНЯТЬ','РЕШЕНИЕ'),1), (('ПРИЗНАТЬ','ВИНОВНАЯ'),2), (('ВЫНЕСТИ','ПРИГОВОР'),3), 
                (('НАЗНАЧИТЬ','НАКАЗАНИЕ'),4), (('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'),5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'),6), (('ОТКЛОНИТЬ','ХОДАТАЙСТВО'),7), (('ОТКЛОНИТЬ','ИСК'),8), 
                (('ОБЖАЛОВАТЬ','ПРИГОВОР'),9), (('ПРЕКРАТИТЬ','ДЕЛО'),10)]

ranks_golden = [p[1] for p in golden_stand]
ranks_lr = [list_no_scores_lr.index(i[0]) for i in golden_stand]
ranks_pmi = [list_no_scores_pmi.index(i[0]) for i in golden_stand]
print('gold', ranks_golden)
print('lr', ranks_lr)
print('pmi', ranks_pmi)

gold [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lr [5, 12, 86, 1992, 4, 3, 1251, 80, 2123, 1648]
pmi [1976, 1963, 1978, 1792, 1924, 1997, 2123, 2062, 1927, 2085]


In [32]:
print('Spearman(lr, prmi)')
print(spearmanr(ranks_lr, ranks_pmi), '\n')
print('Spearman(lr, golden)')
print(spearmanr(ranks_lr,ranks_golden), '\n')
print('Spearman(pmi, golden)')
print(spearmanr(ranks_pmi, ranks_golden))

Spearman(lr, prmi)
SpearmanrResult(correlation=-0.042424242424242413, pvalue=0.90736381781281605) 

Spearman(lr, golden)
SpearmanrResult(correlation=0.45454545454545447, pvalue=0.18690481034933201) 

Spearman(pmi, golden)
SpearmanrResult(correlation=0.43030303030303024, pvalue=0.21449233253280775)
